In [1]:
import tulipy as ti
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_excel("HINDALCO_1D.xlsx")
# Calculate the On-Balance Volume (OBV)
obv = np.where(df['close'].diff() > 0, df['volume'], -df['volume']).cumsum()

min_value = np.min(obv)
max_value = np.max(obv)

# Normalize the array from -1 to 1
obv = 2 * (obv - min_value) / (max_value - min_value)
ema = ti.ema(obv,30)
diff = ti.ema(obv-ema,8)
df['obv'] = diff
# Define the 'signal' column based on the OBV crossing above or below 0
df['signal'] = 0  # Initialize the 'signal' column to 0

# Update 'signal' column based on OBV crossing above or below 0
for i in range(1, len(df)):
    if df['obv'][i] > 0 and df['obv'][i - 1] <= 0:
        df.at[i, 'signal'] = 1
    elif df['obv'][i] < 0 and df['obv'][i - 1] >= 0:
        df.at[i, 'signal'] = -1
def convert_signal(signal):
    if signal == 1:
        return 'Buy'
    elif signal == -1:
        return 'Sell'
    else:
        return 'Hold'

# Apply the function to the 'signal' column to convert the values
df['signal'] = df['signal'].apply(convert_signal)
# Set an initial balance (10000 units of currency)
initial_balance = 10000

# Initialize variables to track balance and positions
balance = initial_balance
position = 0
buy_price = 0

# Define a function to calculate the value of the portfolio
def calculate_portfolio_value(balance, position, current_price):
    return balance + (position * current_price)

# Backtest the strategy
for i in range(len(df)):
    signal = df['signal'][i]
    close_price = df['close'][i]

    if signal == 'Buy' and position == 0:  # Buy signal and no open position
        position = balance // close_price  # Calculate the number of shares to buy (integer division)
        buy_price = close_price
        balance -= (position * close_price)  # Deduct the cost of buying from the balance

    elif signal == 'Sell' and position > 0:  # Sell signal and an open position
        balance += (position * close_price)  # Add the revenue from selling to the balance
        position = 0

# Calculate the final portfolio value
final_value = calculate_portfolio_value(balance, position, df['close'].iloc[-1])

# Calculate the profit/loss
profit_loss = final_value - initial_balance

# Print the results
print(f"Initial Balance: {initial_balance}")
print(f"Final Balance: {final_value}")
print(f"Profit/Loss: {profit_loss}")

